# Import libraries

In [2]:
!pip install scispacy

     |████████████████████████████████| 3.2 MB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 13.0 MB 32.4 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 2.8 MB/s  eta 0:00:01
     |████████████████████████████████| 6.3 MB 40.9 MB/s eta 0:00:01
  Created wheel for scispacy: filename=scispacy-0.2.4-py3-none-any.whl size=35203 sha256=26e1303a97858756434444d441382e77dd082bfaa3360c22e2cdf2f3d805588d
  Stored in directory: /root/.cache/pip/wheels/96/24/54/8c2bf5a6804275431a74bd256821500b3f2911d9ba8470a846
Successfully built scispacy
  Attempting uninstall: rsa
    Found existing installation: rsa 4.0
    Uninstalling rsa-4.0:
      Successfully uninstalled rsa-4.0
  Attempting uninstall: botocore
    Found existing installation: botocore 1.17.6
    Uninstalling botocore-1.17.6:
      Successfully uninstalled botocore-1.17.6


In [3]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_lg-0.2.4.tar.gz

     |████████████████████████████████| 500.6 MB 6.9 kB/s  eta 0:00:01     |█████████                       | 141.8 MB 38.1 MB/s eta 0:00:10
  Created wheel for en-core-sci-lg: filename=en_core_sci_lg-0.2.4-py3-none-any.whl size=501343161 sha256=9e2984dc13c2ac3ea03d8c0cf05b3ec83bac0a012a6aa99203a71c5416090151
  Stored in directory: /root/.cache/pip/wheels/06/a3/b5/bacce7d280488beaf177c3eadbe9f440244201544a0461f6f3
Successfully built en-core-sci-lg


In [1]:
import os
import re
import ast
import string
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import spacy
from spacy.vocab import Vocab
import scispacy
import en_core_sci_lg
from scispacy.abbreviation import AbbreviationDetector
from scispacy.umls_linking import UmlsEntityLinker

from nltk import sent_tokenize

from itertools import combinations
from sklearn.metrics.pairwise import cosine_similarity

from tqdm import tqdm
from numba import jit # parallel processing

# Initialize nlp and test vectors

In [2]:
virus_words = pd.read_csv('/kaggle/input/virus-words/virus_words.txt',header=None)
virus_words[0]

0                     COVID-19
1                     COVID 19
2                    2019-nCoV
3                    2019 nCoV
4                   SARS-CoV-2
5                   SARS-CoV 2
6                   SARS CoV-2
7                   SARS CoV 2
8                    SARS-CoV2
9                   SARS-CoV19
10                SARS-CoV2019
11                    SARSCoV2
12                   SARSCoV 2
13                   SARSCoV19
14                 SARSCoV2019
15                   SARS CoV2
16                  SARS Cov19
17                 SARS Cov 19
18               SARS CoV 2019
19                SARS CoV2019
20            2019 Coronavirus
21    Coronavirus Disease 2019
22            coronavirus 2019
23         novel coronaviruses
24                     covid19
Name: 0, dtype: object

In [3]:
nlp = en_core_sci_lg.load(disable='parser')
linker = UmlsEntityLinker(resolve_abbreviations=True)
nlp.add_pipe(linker)
abbreviation_pipe = AbbreviationDetector(nlp)
nlp.add_pipe(abbreviation_pipe)

new_vector = nlp(
               """Positive-sense single‐stranded ribonucleic acid virus, subgenus 
                   sarbecovirus of the genus Betacoronavirus. 
                   Also known as severe acute respiratory syndrome coronavirus 2, 
                   also known by 2019 novel coronavirus. It is 
                   contagious in humans and is the cause of the ongoing pandemic of 
                   coronavirus disease. Coronavirus disease 2019 is a zoonotic infectious 
                   disease.""").vector

vocab = Vocab()
for virus_word in virus_words[0]:
    nlp.vocab.set_vector(virus_word, new_vector)

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [8]:
# #parser or sentencizer must be enabled
# doc = nlp("the most common symptoms for sars-cov-2 pneumonia at onset of illness were fever and cough, but some patients had presence of dyspnea at a median of 8 days all rights reserved. no reuse allowed without permission. (which was not certified by peer review) is the author/funder, who has granted medrxiv a license to display the preprint in perpetuity. the copyright holder for this preprint this version posted june 3, 2020. . https://doi.org/10.1101/2020.06.01.20119032 doi: medrxiv preprint after onset of illness 9 . dyspnea, usually observed in severe patients with sars-cov-2 pneumonia 10 , may indicate the progression of disease with a low oxygenation index and a state of severe illness. however, up to now, the clinical course of sars-cov-2 pneumonia and risk factors for converting to severe cases remain unknown. this study was performed to investigate clinical characteristics of patients with severe sars-cov-2 pneumonia, and identify risk factors for those with mild to moderate disease who converted to severe cases. 120 patients (36 from ningbo first hospital and 84 from jingzhou central hospital) were included in this study, among which 62 were excluded and 58 were included in the final analysis. the results from this study showed that low peripheral blood lymphocyte count (<1Â·1Ã—10 9 /l) was an independent risk factor for patients converting to severe cases (or: 8Â·0, 95%ci: 1Â·5-41Â·8). with each reduction of 0Â·1Ã—10 9 /l in peripheral blood lymphocyte count, the or for converting to severe cases increased 28% (95%ci, 6%-54%). in addition, using of corticosteroids in mild to moderate patients with sars-cov-2 pneumonia was also associated with a reduced risk of converting to severe cases (or: 0Â·14, 95%ci: 0Â·02-0Â·80). lymphopenia was also observed in most patients with sars-cov infection during their course of illness 11, 12 . he et al believed that lymphopenia was a significant factor of sars-cov infection and lymphocyte counts may be useful in predicting the disease severity and clinical outcomes 12 . a recent study reported an absolute lymphopenia (lymphocyte count <1Â·0Ã—10 9 /l) in 63% of patients with sars-cov-2 pneumonia on admission. low levels of total lymphocyte counts were more marked in those patients in which infection resulted in death 10 . a possible cause for the lymphopenia may be that lymphocytes are directly infected and destroyed by sars-cov-2. however, this requires further study to confirm, as the cellular receptor for sars-cov-2 is angiotensin-converting enzyme 2 13 , which is not expressed on b or t lymphocytes 14 . therefore, depletion of lymphocytes may be secondary to the direct effect of the virus on the lymphocytes or the effect of various cytokine all rights reserved. no reuse allowed without permission. (which was not certified by peer review) is the author/funder, who has granted medrxiv a license to display the preprint in perpetuity. the copyright holder for this preprint this version posted june 3, 2020. . https://doi.org/10.1101/2020.06.01.20119032 doi: medrxiv preprint mediated altered lymphocyte trafficking. since no antiviral treatment for sars-cov-2 infection has been proven to be effective, management of this disease remains clinically based and supportive 15 . considering that acute viral pneumonia is an important cause of acute lung injury (ali), the value of systematic corticosteroids in patients with sars-cov-2 infection is a focus of interest. this study showed patients who received low-to-moderate dose of systematic corticosteroids were less likely to convert to severe cases. in a report by chen et al 16 , the use of systematic corticosteroids in sars resulted in reduced mortality and a shorter hospitalization stay, and was not associated with significant secondary lower respiratory tract infection or other complications. however, it is well known that corticosteroid therapy is a double-edged sword 17, 18 . the immune response may be weak and consequently have more difficulty in eradicating the virus because of treatment with systematic corticosteroids 19 . further studies are urgently required to assess whether systematic corticosteroid treatment is beneficial in patients with sars-cov-2 pneumonia. some limitations of this study should be acknowledged. firstly, this was a retrospective study with a limited sample size, if the conclusion generalized is to be widely used, it would still need a prospective large-scale clinical validation. secondly, different varieties of traditional chinese medicine were administered to most patients, and the effectiveness and potential adverse effects of those drugs given remain largely unknown. thirdly, examinations of lymphocyte subsets were undertaken in this study, and in addition, total lymphocyte counts were not observed dynamically. despite the above limitations, we believe that our study has shown important and novel findings about the predictors of severe cases in mild to moderate patients with sars-cov-2 pneumonia. to our knowledge, this is the first study that has demonstrated that lymphocyte count is a useful predictor for severity of sars-cov-2 pneumonia. this may help clinicians more accurately predict prognosis, and triage all rights reserved. no reuse allowed without permission. (which was not certified by peer review) is the author/funder, who has granted medrxiv a license to display the preprint in perpetuity. the copyright holder for this preprint this version posted june 3, 2020. . https://doi.org/10.1101/2020.06.01.20119032 doi: medrxiv preprint priorities to improve clinical outcomes. the findings from this study showed that low lymphocyte count (<1Â·1Ã—10 9 /l) in peripheral blood was an independent risk factor for patients who converted to severe cases. in addition, using of systematic corticosteroids in mild to moderate patients with sars-cov-2 pneumonia was associated with a reduced risk of converting to severe cases. further prospective studies are warranted to confirm these findings.")
# sents = [str(sent) for sent in doc.sents]
# sents.index('despite the above limitations, we believe that our study has shown important and novel findings about the predictors of severe cases in mild to moderate patients with sars-cov-2 pneumonia.')
# for sent in doc.sents:
#     if str(sent) == 'despite the above limitations, we believe that our study has shown important and novel findings about the predictors of severe cases in mild to moderate patients with sars-cov-2 pneumonia.':
#         print(sent.vector)
#         v1 = sent.vector

ValueError: [E030] Sentence boundaries unset. You can add the 'sentencizer' component to the pipeline with: nlp.add_pipe(nlp.create_pipe('sentencizer')) Alternatively, add the dependency parser, or set sentence boundaries by setting doc[i].is_sent_start.

In [6]:
# #parser or sentencizer must be enabled
# doc = nlp('despite the above limitations, we believe that our study has shown important and novel findings about the predictors of severe cases in mild to moderate patients with sars-cov-2 pneumonia.')
# for sent in doc.sents:
#     print(sent.vector)
#     v2 = sent.vector

[-0.0445595  -0.02728982  0.08810052 -0.0202131   0.0080669  -0.02247757
 -0.03786988 -0.00330973  0.00700686 -0.00102334  0.03151169 -0.1250888
  0.05129229 -0.0407907   0.00537252 -0.05280348 -0.03947191 -0.13587129
  0.01905933  0.09522764  0.06247263 -0.04573615 -0.01230669 -0.10526265
  0.06168577  0.02632068  0.04959745 -0.09677126 -0.01858239  0.04353651
  0.0020929  -0.11511589 -0.02223008 -0.04489174  0.13471201 -0.01841085
  0.01872057  0.11095951 -0.1027195  -0.03613376 -0.06312553  0.01613808
  0.04632265 -0.01602972  0.05645413 -0.04484621 -0.10683692 -0.11962598
  0.02976297 -0.19699678  0.00736201  0.02555153 -0.03574163  0.05970762
 -0.03827594 -0.0290972  -0.05108489  0.0165926  -0.00119462 -0.07227673
  0.0182394  -0.00743266 -0.00647731  0.07168672  0.05977864  0.03197805
 -0.00620746  0.08659254  0.06747445  0.04313469  0.0002598  -0.08746701
 -0.02293878  0.00041063 -0.01194189 -0.0242598   0.14824276  0.0164418
  0.0280341   0.00734268  0.00241804 -0.10050812  0.0

In [7]:
# if all([True for i,v in enumerate(v1) if v2[i]==v]):
#     print('vectors are the same')

vectors are the same


# Define i/o paths

In [4]:
input_data_path = '/kaggle/input/v31-claims/'
input_data_file = 'claims_flag_each_cord_uid_200620.csv'

metadata_file_path = '/kaggle/input/CORD-19-research-challenge/'
metadata_file = 'metadata.csv'

# Read and process data

In [5]:
metadata = pd.read_csv(metadata_file_path+metadata_file)
metadata = metadata[['cord_uid','publish_time','title']]

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,6,13,14,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
input_data = pd.read_csv(input_data_path+input_data_file)
no_claims_cord_uid = set(input_data.loc[input_data.claim_flag==0,'cord_uid'])\
                               - set(input_data.loc[input_data.claim_flag==1,'cord_uid'])

print('No of unique sentences in claims:',input_data.loc[input_data.claim_flag==1,'claims'].nunique())
print('No of rows in claims:',len(input_data.loc[input_data.claim_flag==1,:]))
print('\n')
print('Total no of papers:',input_data.cord_uid.nunique())
print('Papers with claims:',input_data.loc[input_data.claim_flag==1,'cord_uid'].nunique())
print('Papers with no claims:',len(no_claims_cord_uid))

No of unique sentences in claims: 4559
No of rows in claims: 4781


Total no of papers: 2564
Papers with claims: 1819
Papers with no claims: 745


In [26]:
#For papers with no claims, tokenize to sentences
claims_data = input_data.loc[input_data.claim_flag==1,:].copy().reset_index(drop=True)
no_claims_data = input_data.loc[input_data.cord_uid.isin(no_claims_cord_uid),:].copy().reset_index(drop=True)

dict = {}
k = 0

for i,text in enumerate(no_claims_data.text):
    for sent in sent_tokenize(text):
        dict[k] = {'cord_uid':no_claims_data.cord_uid[i],\
                   'section':no_claims_data.section[i],\
                   'text':no_claims_data.text[i],\
                   'drug_terms_used':no_claims_data.drug_terms_used[i],\
                   'claims':sent}
        k = k + 1
        
no_claims_data = pd.DataFrame.from_dict(dict, "index")
print('Total rows for papers with no claims:',len(no_claims_data))

Total rows for papers with no claims: 19922


In [27]:
print('Papers with at least 1 claim in abstract section:',claims_data.loc[claims_data.section.str.lower()=='abstract','cord_uid'].nunique())

Papers with at least 1 claim in abstract section: 527


In [14]:
claims_data.section.unique()

array(['Abstract', 'CONCLUSION', 'CONCLUSION:', 'Discussion',
       'Conclusion', 'DISCUSSION', 'CONCLUSIONS',
       'Factors That Put People With Schizophrenia at Elevated Risk of Acquiring COVID-19 and of Experiencing Poor Outcomes',
       'Conclusions', 'Discussion and Conclusion', '4. Conclusions',
       '8. Conclusions', 'Results and Discussion',
       'Complications, treatment and clinical outcomes during hospitalization and follow-up 296',
       '4. Discussion', '5. Conclusions', 'CONCLUSION 40',
       'Table: Demographics, clinical characteristics at admission, treatment, and outcomes of five patients with HIV and COVID-19',
       'Results and discussion', 'Results and Discussion.',
       'RESULTS AND DISCUSSION', 'Added value of this study',
       'The proposed D-peptide ::: Discussion and conclusions',
       'The proposed L-peptide ::: Discussion and conclusions',
       '| DISCUSSION', 'Discussion and conclusions', 'Discussion:',
       'DISCUSSION AND CONCLUSIONS

In [32]:
claims_data

,cord_uid,section,text,drug_terms_used,claims,claim_flag
0,01es0zv4,Abstract,coronavirus disease 2019 has become a global p...,"chloroquine,lopinavir,remdesivir,ritonavir,azi...",it is important to have a knowledge of potenti...,1.0
1,01es0zv4,CONCLUSION,covid-19 is a pandemic with high morbidity and...,"chloroquine,lopinavir,remdesivir,ritonavir,azi...",caution and careful monitoring should be exerc...,1.0
2,01es0zv4,CONCLUSION:,covid-19 is a pandemic with high morbidity and...,"chloroquine,lopinavir,remdesivir,ritonavir,azi...",caution and careful monitoring should be exerc...,1.0
3,02q9y011,Abstract,"the virus that causes covid-19, sars-cov-2, ha...",amino acids,although 26% of the amino acids in this sars-c...,1.0
4,033q671f,Discussion,the most common symptoms for sars-cov-2 pneumo...,corticosteroids,"despite the above limitations, we believe that...",1.0
...,...,...,...,...,...,...
4776,zsz9xnxt,Discussion,we demonstrate that mechanically ventilated pa...,carbon dioxide,we demonstrate that mechanically ventilated pa...,1.0
4777,zsz9xnxt,Discussion,we demonstrate that mechanically ventilated pa...,carbon dioxide,"in conclusion, we show that higher peep levels...",1.0
4778,zsz9xnxt,Discussion,we demonstrate that mechanically ventilated pa...,carbon dioxide,we suggest using prone position for an extende...,1.0
4779,zsz9xnxt,Discussion,we demonstrate that mechanically ventilated pa...,carbon dioxide,we suggest using prone position for an extende...,1.0


In [48]:
no_claims_data

,cord_uid,section,sentence,drug_terms_used,claims
0,019rcbpg,Potential biological mechanisms of SARS-CoV-2 ...,there are indications in the literature of a n...,ifn-gamma,there are indications in the literature of a n...
1,019rcbpg,Potential biological mechanisms of SARS-CoV-2 ...,there are indications in the literature of a n...,ifn-gamma,evidence of a variety of neurological symptoms...
2,019rcbpg,Potential biological mechanisms of SARS-CoV-2 ...,there are indications in the literature of a n...,ifn-gamma,"(kotfis, 2020) interestingly two case studies ..."
3,019rcbpg,Potential biological mechanisms of SARS-CoV-2 ...,there are indications in the literature of a n...,ifn-gamma,"(moriguchi, 2020) ace2 is a functional recepto..."
4,019rcbpg,Potential biological mechanisms of SARS-CoV-2 ...,there are indications in the literature of a n...,ifn-gamma,"(netland et al., 2008) taken together this ind..."
...,...,...,...,...,...
19917,zwqci59h,Discussion,although recent studies have reported epidemio...,"compro,cyclosporine,methylprednisolone,prednis...",2019jyjh09) and the national natural science f...
19918,zwqci59h,Discussion,although recent studies have reported epidemio...,"compro,cyclosporine,methylprednisolone,prednis...","81874091, 81974396, 81772724, and 81672529)."
19919,zwqci59h,Discussion,although recent studies have reported epidemio...,"compro,cyclosporine,methylprednisolone,prednis...",conflicts of interest: the authors have nothin...
19920,zwqci59h,Discussion,although recent studies have reported epidemio...,"compro,cyclosporine,methylprednisolone,prednis...",acknowledgments: we thank all the patients and...


In [49]:
no_claims_data.to_csv('no_claims_tokenized.csv')

In [9]:
drug_terms = []
for drugs in input_data.drug_terms_used:
    drug_terms = drug_terms + str(drugs).split(',')
drug_terms = list(set(drug_terms))
drug_terms.append('acei/arb')
len(drug_terms)

1006

# Replace drug name short forms with full forms

In [28]:
dict = {'hcq':'hydroxychloroquine','cq':'chloroquine','azt':'azithromycin','azi':'azithromycin', 'az':'azithromycin'}
for key,value in dict.items():
    claims_data['claims'] = [t.lower().replace(key,value) for t in claims_data.claims]
    no_claims_data['claims'] = [t.lower().replace(key,value) for t in no_claims_data.claims]

# Append sentences from papers with no claims to claim sentences 

In [29]:
claims_data = claims_data.append(no_claims_data).reset_index(drop=True)

In [30]:
del(no_claims_data)

# Filter to sentences that mention drug names

In [58]:
sentences_to_keep = [any([True for d in drug_terms if d in c]) for c in claims_data.claims]
claims_data = claims_data[sentences_to_keep].reset_index(drop=True)
print(len(claims_data))

claims_data['drug_terms_mention'] = [[d for d in drug_terms if d in c] for c in claims_data.claims]

4643


In [59]:
print('No of papers being compared:',claims_data.cord_uid.nunique())

No of papers being compared: 1383


# Pair similar claims

In [61]:
paper_pairs = list(combinations(claims_data.index,2))
paper_pairs_filt = []
for i,j in tqdm(paper_pairs):
    if claims_data.cord_uid[i]!=claims_data.cord_uid[j]:
        if any(d1 in claims_data.drug_terms_mention[i] for d1 in claims_data.drug_terms_mention[j]):
            paper_pairs_filt.append((i,j))
len(paper_pairs_filt)

100%|██████████| 10776403/10776403 [29:28<00:00, 6095.01it/s]


721705

In [63]:
claims_data['w2vVector'] = [nlp(c).vector.reshape(1,-1) for c in claims_data.claims]

In [66]:
dict = {}
k = 0

jit(nopython=True, parallel=True)

for i,j in tqdm(paper_pairs_filt):
    #cos_sim = cosine_similarity(np.array(claims_data.w2vVector[i]).reshape(1,-1),np.array(claims_data.w2vVector[j]).reshape(1,-1))[0][0]
    cos_sim = cosine_similarity(claims_data.w2vVector[i],claims_data.w2vVector[j])[0][0]
    if cos_sim>=0.5:
        dict[k] = {'paper1_cord_uid':claims_data.cord_uid[i],\
                    'paper2_cord_uid':claims_data.cord_uid[j],\
                    'text1':claims_data.claims[i],\
                    'text2':claims_data.claims[j],\
                    'similarity_score':cos_sim,\
                    'drugs1':claims_data.drug_terms_used[i],\
                    'drugs2':claims_data.drug_terms_used[j]}
        k = k + 1
claims_similarity = pd.DataFrame.from_dict(dict, "index")

100%|██████████| 721705/721705 [08:58<00:00, 1338.99it/s]


In [67]:
print(len(claims_similarity))

650019


In [68]:
print(claims_similarity.paper1_cord_uid.nunique())
print(claims_similarity.paper2_cord_uid.nunique())

1288
1230


# Add title & publish time fields

In [69]:
claims_similarity = pd.merge(claims_similarity,metadata,how='inner',\
                           left_on = 'paper1_cord_uid',\
                          right_on = 'cord_uid')
cols_rename = {'title':'title1','publish_time':'publish_time1'}
claims_similarity.rename(columns = cols_rename,inplace=True)

claims_similarity = pd.merge(claims_similarity,metadata,how='inner',\
                           left_on = 'paper2_cord_uid',\
                          right_on = 'cord_uid')
cols_rename = {'title':'title2','publish_time':'publish_time2'}
claims_similarity.rename(columns = cols_rename,inplace=True)

print(len(claims_similarity))

660964


In [70]:
claims_similarity = claims_similarity.drop(columns=['cord_uid_x','cord_uid_y'])

In [71]:
claims_similarity.to_csv('drug_individual_claims_filtered.csv')

In [ ]:
claims_similarity.to_json('drug_individual_claims_filtered.json')